In [1]:
!pip list | grep lang

langchain                0.0.263
langchainplus-sdk        0.0.17
langsmith                0.0.22


In [2]:
!pip list | grep openai

openai                   0.27.8


### LangChain Expression Language
In this notebook we will have a try on the latest feature released by langchain, using pipeline to link model, prompt and any function or parser you want.


#### 1. Toy example
We will first take a look at a simplest example to link a LLM and a prompt. Before we take a look at Langchain expression language, let's take a look at pipe package which support pipe operation in python

In [18]:
from pipe import select, where

numbers = [1, 2, 3, 4, 5]
# find even numbers and multiple by 3
results = list(numbers |  where(lambda x: x % 2 == 0) | select(lambda x: x * 3))

results

[6, 12]

In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv

# load env for llm service
load_dotenv()

True

In [24]:
# create llm
llm = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')

# create prompt template
prompt = ChatPromptTemplate.from_template("Show me description for company {company_name} in 100 words.")

# chain model and prompt
chain = prompt | llm

chain

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[], last=AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api

In [25]:
chain.invoke({"company_name": "HSBC"})

AIMessage(content='HSBC is a multinational banking and financial services company headquartered in London, UK. It operates in over 60 countries worldwide, providing a wide range of banking and financial products and services to individuals, businesses, and institutions. HSBC is one of the largest banks in the world, with a strong presence in Asia, Europe, and the Americas. The company offers services such as personal and business banking, wealth management, insurance, and investment banking. HSBC is committed to sustainability and responsible business practices, and has implemented initiatives to reduce its carbon footprint and support economic growth in developing countries.', additional_kwargs={}, example=False)

In [26]:
# a chain with a string output parser
chain_with_output_parser = prompt | llm | StrOutputParser()
chain_with_output_parser

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api_version

In [28]:
chain_with_output_parser.invoke({"company_name": "HSBC"})

'HSBC Holdings plc is a multinational banking and financial services company headquartered in London, United Kingdom. It is one of the largest banking organizations in the world, with a presence in 64 countries and territories, serving more than 40 million customers. HSBC offers a wide range of financial products and services, including retail and commercial banking, wealth management, insurance, and investment banking. The company is committed to sustainability and social responsibility, with initiatives focused on climate change, education, and community development. HSBC is listed on the London, Hong Kong, and New York stock exchanges.'

#### 2. Attaching Function Call information

We are not call the function acctually here, we are just using the function specification and let LLM to figure out the input parameter for the function

In [32]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# functions = [
#     {
#       "name": "save_description",
#       "description": "save the description for the a company",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "company_name": {
#             "type": "string",
#             "description": "The name of the company"
#           },
#           "description": {
#             "type": "string",
#             "description": "The description of the company"
#           }
#         },
#         "required": ["company_name", "description"]
#       }
#     }
#   ]


# chain = (
#     prompt 
#     | llm.bind(function_call= {"name": "save_description"}, functions= functions) 
#     | JsonOutputFunctionsParser()
# )

# chain.invoke({"company_name": "HSBC"})

The output should be look like, bue since our OPENAI model at Azure is still the old GPT 3.5 which does not support functional call and it will encounter an error.

{ 
  'company_name' : 'HSBC',
  'description' : 'HSBC Holdings plc is a multinational banking and financial services company headquartered in London, United Kingdom. It is one of the largest banking organizations in the world, with a presence in 64 countries and territories, serving more than 40 million customers. HSBC offers a wide range of......'
}

### 3. Runnable in Langchain

LangChain Expression Language (LEL) provides a convenient and straightforward approach for writing components in a declarative manner and effortlessly combining them. To gain a better understanding of its functionality, let's examine its runnable interface.

LangChain introduces a runnable interface that serves as a blueprint for implementing various components. Subclasses of this interface are required to implement the following functions:
- stream: handles streaming output
- invoke: processes single input
- batch: handles batch input
- astream: handles asynchronous streaming
- ainvoke: performs asynchronous invocation
- abatch: handles asynchronous batch processing

In [33]:
from langchain.schema.runnable import Runnable

# the ChatPromptTemplate implemented Runnable as well
print(isinstance(prompt, Runnable))

True


In [42]:
# let's write our own component

from langchain.load.serializable import Serializable
from langchain.schema.runnable import RunnableConfig, RunnablePassthrough
from langchain.schema.runnable.base import Input, Runnable, Optional


class StdOutputRunnable(Serializable, Runnable[Input, Input]):

    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Input, config: Optional[RunnableConfig] = None) -> Input:
        print(f"I recevied input ->  {input} ")
        return self._call_with_config(lambda x: x, input, config)


runnable_chain =  RunnablePassthrough() | StdOutputRunnable()

runnable_chain.invoke({'company_name':'HSBC'})

I recevied input ->  {'company_name': 'HSBC'} 


{'company_name': 'HSBC'}

In [43]:
chain = prompt | StdOutputRunnable() | llm

chain.invoke({'company_name': 'Alibaba'})

I recevied input ->  messages=[HumanMessage(content='Show me description for company Alibaba in 100 words.', additional_kwargs={}, example=False)] 


AIMessage(content="Alibaba Group Holding Limited is a multinational conglomerate founded by Jack Ma in 1999. It is one of the world's largest e-commerce and technology companies, with businesses spanning across e-commerce, cloud computing, digital media, entertainment, and more. Alibaba's main platform, Taobao, is the world's largest online marketplace, while Tmall is China's largest business-to-consumer platform. The company also operates AliExpress, a global online retail platform, and Alipay, a popular mobile payment system. With a strong focus on innovation and technology, Alibaba has become a major player in the global digital economy, serving millions of customers worldwide.", additional_kwargs={}, example=False)

#### 4. Retriever

In [56]:
# from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.schema.runnable import RunnablePassthrough

# # create embedding model
# embeddings = OpenAIEmbeddings(
#     deployment="text-embedding-ada-002",
#     model="text-embedding-ada-002"
# )

# # Create the retriever
# vectorstore = Chroma.from_texts(["harrison worked at kensho"], embedding=embeddings)
# retriever = vectorstore.as_retriever()

In [57]:
#!pip install chromadb

In [47]:
!export HNSWLIB_NO_NATIVE=1

In [53]:
!pip install --upgrade setuptools

  Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/c7/42/be1c7bbdd83e1bfb160c94b9cafd8e25efc7400346cf7ccdbdb452c467fa/setuptools-68.0.0-py3-none-any.whl.metadata
  Using cached setuptools-68.0.0-py3-none-any.whl.metadata (6.4 kB)
Using cached setuptools-68.0.0-py3-none-any.whl (804 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0


In [55]:
!clang -v

Apple clang version 12.0.0 (clang-1200.0.31.1)
Target: x86_64-apple-darwin22.5.0
Thread model: posix
InstalledDir: /Library/Developer/CommandLineTools/usr/bin
